<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-26 10:22:29
-------------------
qualified stocks: 93
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.39 L
Current:  1.36 C
-------------------
Today PnL: -76.38 K (-0.56%)
Current PnL: -18.93 L (-12.88%)
CY Booked + Current PnL: -10.20 L (-6.94%)
-------------------
Total profit:  3.39 L
Total loss:  -22.32 L
-------------------
Total Booked + Current PnL: 16.78 L (13.98%)
Total Booked PnL: 35.71 L (29.75%)
Curr Year Booked PnL: 8.73 L (6.4%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 78.04 L (57.22%)
Deployed:  1.20 C
Current:  1.36 C
CAGR/XIRR %: 8.06%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-18.25,H-LC,7.38,113264.0,8952.0,7702.0,-1.11,8.58,6.80,15.96,9.0,1.16,0.85,26.93,X5K,ATH,METALS
37,HINDUNILVR,2413.81,2723.00,-5.43,H-LC,8.15,297203.0,26856.0,7787.0,0.84,9.93,2.62,12.81,24.0,3.45,2.23,24.30,XY25,NTT,FMCG
4,APOLLOHOSP,7098.95,8285.00,-14.19,H-LC,8.11,171457.0,15280.0,10819.0,-0.73,9.78,6.31,16.71,22.0,1.41,1.29,28.93,X40N,BTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,12.48,H-MC,14.62,160567.0,34367.0,11802.0,-0.87,27.23,7.35,36.59,108.0,2.91,1.21,43.21,XY24,NTT,HEALTHCARE
68,SBIN,760.30,863.00,-13.25,M-LC,4.06,215684.0,14204.0,13006.0,-0.31,7.05,6.03,13.51,60.0,1.09,1.62,20.53,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,KPIGREEN,497.21,731.35,5.57,H-SC,11.09,125887.0,590.0,58412.0,-2.05,0.47,46.40,47.09,141.0,0.01,0.95,57.56,MH,ATH,POWER
61,RAJOOENG,102.50,143.33,-32.41,H-SC,10.05,102430.0,-70.0,40900.0,-1.46,-0.07,39.93,39.83,114.0,-0.00,0.77,1.42,AR,ATH,MISC
56,NATIONALUM,189.63,247.44,-49.46,H-MC,1.93,100424.0,-80.0,30720.0,-0.49,-0.08,30.59,30.49,79.0,-0.00,0.76,32.30,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-39.61,H-LC,3.09,145363.0,-4569.0,19159.0,-0.00,-3.05,13.18,9.74,4.0,-0.24,1.09,3.00,X40,NTT,FMCG
53,LTIM,5564.16,7230.2,-4.05,H-LC,3.13,190440.0,-9870.0,69853.0,-0.06,-4.93,36.68,29.94,16.0,-0.14,1.43,31.57,X200,ATH,IT
57,NESTLEIND,1134.30,1377.0,-11.15,H-LC,3.40,273312.0,7886.0,48896.0,1.30,2.97,17.89,21.40,11.0,0.16,2.05,9.81,XY25,NTT,FMCG
21,CIPLA,1492.70,1795.0,-18.41,H-LC,4.23,217255.0,12755.0,28656.0,-0.96,6.24,13.19,20.25,10.0,0.45,1.63,15.51,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-13.45,H-LC,6.42,168858.0,-32528.0,120142.0,-0.39,-16.15,71.15,43.51,27.0,-0.27,1.27,17.54,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,27.28,84518.0,-29031.0,69085.0,2.40,-25.57,81.74,35.27,268.0,-0.42,0.64,106.86,XR,NTT,HOTELS
16,BRITANNIA,5502.66,6362.67,12.90,H-LC,8.66,204228.0,6132.0,24834.0,2.21,3.10,12.16,15.63,26.0,0.25,1.54,25.62,XY25,ATH,FMCG
57,NESTLEIND,1134.30,1377.00,-11.15,H-LC,3.40,273312.0,7886.0,48896.0,1.30,2.97,17.89,21.40,11.0,0.16,2.05,9.81,XY25,NTT,FMCG
35,HEROMOTOCO,4311.81,5949.07,-3.47,H-MC,14.09,179389.0,28476.0,28828.0,1.22,18.87,16.07,37.97,93.0,0.99,1.35,48.95,AR,ATH,AUTO
79,TATAMOTORS,761.77,1065.00,-20.84,M-LC,4.88,249308.0,-27215.0,137294.0,0.96,-9.84,55.07,39.81,54.0,-0.20,1.87,19.48,XY24,NTT,AUTO


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,STARHEALTH,551.56,761.00,5.34,H-SC,2.17,156725.0,-43491.0,119518.0,-2.65,-21.72,76.26,37.97,144.0,-0.36,1.18,25.84,XY24,NTT,INSURANCE
40,ICICIGI,1839.64,2260.25,-19.41,H-MC,5.06,138432.0,2299.0,28822.0,-2.52,1.69,20.82,22.86,91.0,0.08,1.04,14.51,X40,ATH,INSURANCE
29,EASEMYTRIP,18.12,26.40,-12.72,M-SC,28.41,84016.0,-92799.0,173594.0,-2.27,-52.48,206.62,45.70,197.0,-0.53,0.63,0.00,XY24,NTT,TRAVEL
8,AWL,325.39,485.00,-65.97,H-MC,6.65,209385.0,-59062.0,190750.0,-2.25,-22.00,91.10,49.05,116.0,-0.31,1.57,7.66,XY24,NTT,FMCG
71,SIEMENS,4406.90,7969.85,-3.68,H-LC,33.17,77332.0,-32840.0,121914.0,-2.25,-29.81,157.65,80.85,15.0,-0.27,0.58,13.90,AR,ATH,ELECTRICAL


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,97.23,M-SC,8.61,84947.0,-15830.0,15919.0,-1.17,-15.71,18.74,0.09,245.0,-0.99,0.64,11.60,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,84.36,H-SC,15.47,116505.0,-25974.0,45495.0,-1.37,-18.23,39.05,13.70,163.0,-0.57,0.88,37.51,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-19.33,H-SC,3.02,115092.0,-28711.0,55463.0,-1.13,-19.97,48.19,18.60,149.0,-0.52,0.87,27.01,OX40N,NTT,CERAMICS
50,KANSAINER,299.63,340.0,-69.25,H-SC,3.63,212166.0,-57501.0,93841.0,0.51,-21.32,44.23,13.47,138.0,-0.61,1.60,7.31,XY24,NTT,PAINTS
43,INDIGOPNTS,1407.73,1408.0,108.71,M-SC,1.72,136908.0,-37651.0,37677.0,-1.98,-21.57,27.52,0.02,221.0,-1.00,1.03,19.09,OX40N,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-25.80,H-LC,15.93,245014.0,-50920.0,144828.0,0.02,-17.21,59.11,31.73,1.0,-0.35,1.84,4.60,X40,BTT,IT
45,INFY,1461.46,2275.00,-16.01,H-LC,11.88,274676.0,13075.0,132559.0,0.16,5.00,48.26,55.67,3.0,0.10,2.07,11.41,X40,BTT,IT
47,ITC,411.90,452.00,-39.61,H-LC,3.09,145363.0,-4569.0,19159.0,-0.00,-3.05,13.18,9.74,4.0,-0.24,1.09,3.00,X40,NTT,FMCG
89,VBL,492.64,671.64,-9.65,H-LC,8.64,274541.0,9501.0,86810.0,-1.10,3.58,31.62,36.33,5.0,0.11,2.06,17.36,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.42,H-LC,12.56,198920.0,-17599.0,110500.0,0.74,-8.13,55.55,42.91,7.0,-0.16,1.50,4.17,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,12.48,H-MC,14.62,160567.0,34367.0,11802.0,-0.87,27.23,7.35,36.59,108.0,2.91,1.21,43.21,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-6.62,H-LC,11.20,233230.0,48749.0,21620.0,0.73,26.42,9.27,38.14,38.0,2.25,1.75,39.83,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,12.48,H-MC,14.62,160567.0,34367.0,11802.0,-0.87,27.23,7.35,36.59,108.0,2.91,1.21,43.21,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,9.32,H-MC,4.79,188770.0,24100.0,43870.0,-0.88,14.64,23.24,41.28,89.0,0.55,1.42,23.79,X40N,ATH,AC
51,KPIGREEN,497.21,731.35,5.57,H-SC,11.09,125887.0,590.0,58412.0,-2.05,0.47,46.40,47.09,141.0,0.01,0.95,57.56,MH,ATH,POWER
90,VOLTAS,1278.28,1918.49,-3.32,H-MC,6.74,202905.0,11163.0,84875.0,-0.35,5.82,41.83,50.08,99.0,0.13,1.53,13.02,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-3.47,H-MC,14.09,179389.0,28476.0,28828.0,1.22,18.87,16.07,37.97,93.0,0.99,1.35,48.95,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.47,H-MC,14.09,179389.0,28476.0,28828.0,1.22,18.87,16.07,37.97,93.0,0.99,1.35,48.95,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-49.79,H-SC,12.38,140370.0,17034.0,116732.0,0.41,13.81,83.16,108.45,119.0,0.15,1.06,39.25,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-42.94,M-SC,5.56,96488.0,4990.0,73765.0,-1.72,5.45,76.45,86.07,223.0,0.07,0.73,39.58,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-10.16,M-LC,7.72,157517.0,6572.0,102890.0,0.09,4.35,65.32,72.51,53.0,0.06,1.18,10.64,XR,NTT,IT
51,KPIGREEN,497.21,731.35,5.57,H-SC,11.09,125887.0,590.0,58412.0,-2.05,0.47,46.40,47.09,141.0,0.01,0.95,57.56,MH,ATH,POWER


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,27.28,84518.0,-29031.0,69085.0,2.40,-25.57,81.74,35.27,268.0,-0.42,0.64,106.86,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-3.16,H-SC,16.17,90192.0,-10771.0,105299.0,-1.18,-10.67,116.75,93.63,148.0,-0.10,0.68,34.34,SR,ATH,CHEMICALS
54,MASFIN,326.60,398.61,-19.15,H-SC,12.53,91920.0,-6060.0,27659.0,-1.97,-6.18,30.09,22.05,152.0,-0.22,0.69,33.61,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-18.25,H-LC,7.38,113264.0,8952.0,7702.0,-1.11,8.58,6.80,15.96,9.0,1.16,0.85,26.93,X5K,ATH,METALS
84,UJJIVANSFB,52.77,60.00,84.36,H-SC,15.47,116505.0,-25974.0,45495.0,-1.37,-18.23,39.05,13.70,163.0,-0.57,0.88,37.51,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,27.28,84518.0,-29031.0,69085.0,2.40,-25.57,81.74,35.27,268.0,-0.42,0.64,106.86,XR,NTT,HOTELS
28,DMART,3764.92,5201.00,-6.62,H-LC,11.20,233230.0,48749.0,21620.0,0.73,26.42,9.27,38.14,38.0,2.25,1.75,39.83,X40N,NTT,FMCG
35,HEROMOTOCO,4311.81,5949.07,-3.47,H-MC,14.09,179389.0,28476.0,28828.0,1.22,18.87,16.07,37.97,93.0,0.99,1.35,48.95,AR,ATH,AUTO
37,HINDUNILVR,2413.81,2723.00,-5.43,H-LC,8.15,297203.0,26856.0,7787.0,0.84,9.93,2.62,12.81,24.0,3.45,2.23,24.30,XY25,NTT,FMCG
15,BLUESTARCO,1646.70,2326.38,9.32,H-MC,4.79,188770.0,24100.0,43870.0,-0.88,14.64,23.24,41.28,89.0,0.55,1.42,23.79,X40N,ATH,AC


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.98
1,25,41.69
2,50,71.29


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.21
LC    36.49
MC    26.33
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.35
X40      14.42
X40N     13.95
XY25     11.90
XR        9.02
AR        8.47
OX40N     7.40
X5K       2.27
MH        1.71
X200      1.43
SR        1.11
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.24
H-MC    23.44
H-SC    22.51
M-SC    13.14
M-LC     7.21
M-MC     2.56
L-SC     1.56
L-LC     1.04
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.65,0.27,30.27
IT,12.12,-15.82,73.66
FINANCE,7.99,-22.76,68.15
BANKS,7.48,-14.78,63.52
MISC,5.88,-17.62,85.05
PAINTS,5.52,-19.13,42.61
ELECTRICAL,4.75,-11.14,59.27
AUTO,4.58,-10.80,59.01
AC,3.69,1.07,32.43


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2867506.0,23
XR,1093985.0,13
AR,1082326.0,10
X40,816956.0,11
X40N,510803.0,11
OX40N,477132.0,10
XY25,463875.0,8
SR,195946.0,2
X5K,136875.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2785310.0,27
M-SC,1497835.0,17
H-MC,1262032.0,18
H-LC,1143755.0,19
M-LC,418821.0,5
M-MC,337227.0,2
L-SC,254553.0,3
L-MC,59150.0,1
L-LC,45706.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      943027.0      6
M-SC       XY24      876670.0      7
H-SC       AR        661924.0      5
           XR        526898.0      6
H-LC       X40       510772.0      6
H-MC       XY24      487908.0      5
M-MC       XY24      337227.0      2
H-LC       AR        232414.0      2
H-MC       X40       222323.0      4
H-SC       OX40N     206541.0      4
H-MC       XY25      199267.0      2
H-SC       SR        195946.0      2
H-LC       X40N      192575.0      5
H-SC       X40N      192562.0      3
M-LC       XY24      173752.0      2
L-SC       XR        164725.0      2
M-SC       AR        159160.0      2
           OX40N     130504.0      4
M-LC       X5K       129173.0      1
H-MC       X40N      125666.0      3
M-SC       XY25      124379.0      1
           XR        123261.0      2
H-MC       XR        117061.0      1
M-LC       XR        102890.0      1
L-SC       OX40N      89828.0      1
M-SC       X40        83861.0      1
H-LC       XY25       81517.0      3
           X200       69853.0      1
L-MC       XR         59150.0      1
H-SC       MH         58412.0      1
H-MC       OX40N      50259.0      1
H-LC       XY24       48922.0      1
L-LC       XY25       45706.0      1
H-MC       MH         30720.0      1
           AR         28828.0      1
M-LC       XY25       13006.0      1
H-LC       X5K         7702.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
